<a href="https://colab.research.google.com/github/sogand73/OCT-Classification/blob/main/oct_classification_using_yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import MiniBatchDictionaryLearning

In [2]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle datasets download paultimothymooney/kermany2018/

100% 10.8G/10.8G [02:00<00:00, 169MB/s]
100% 10.8G/10.8G [02:00<00:00, 96.6MB/s]


In [4]:
!unzip kermany2018.zip

Streaming output truncated to the last 5000 lines.
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8050636-2.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055145-1.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055145-2.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055145-3.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055590-1.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055590-2.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8055590-3.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8056259-1.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8056259-2.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8056259-3.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8056259-4.jpeg  
  inflating: oct2017/__MACOSX/OCT2017 /train/NORMAL/._NORMAL-8056259-5.jpeg  
  inflating: 

In [7]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 695.2/695.2 kB 6.3 MB/s eta 0:00:00


In [103]:
shutil.rmtree('/content/oct_my_dataset')

In [104]:
import os
!mkdir '/content/oct_my_dataset'
!mkdir '/content/oct_my_dataset/train'
!mkdir '/content/oct_my_dataset/test'
!mkdir '/content/oct_my_dataset/val'


In [105]:
classes = ['NORMAL', 'DME', 'DRUSEN', 'CNV']
for subfolder in classes:
  train_subfolder = os.path.join('/content/oct_my_dataset/train/',subfolder)
  test_subfolder = os.path.join('/content/oct_my_dataset/test/',subfolder)
  val_subfolder = os.path.join('/content/oct_my_dataset/val/',subfolder)
  os.makedirs(train_subfolder, exist_ok=True)
  os.makedirs(test_subfolder, exist_ok=True)
  os.makedirs(val_subfolder, exist_ok=True)

In [106]:
import shutil


# Define source and destination directories
dataset_folder = '/content/OCT2017 '  # Replace with your source folder path

destination_folder = '/content/oct_my_dataset/'  # Replace with your destination folder path

for folder in [['train/',500], ['test/',100], ['val/',8]]:
  for subfolder in ['NORMAL', 'DME', 'DRUSEN', 'CNV']:
    # List files in the source folder
    source_folder = os.path.join(dataset_folder, folder[0], subfolder) + '/'
    files_to_copy = os.listdir(source_folder)

    # Copy each image file from the source folder to the destination folder
    for file_name in files_to_copy[:folder[1]]:
        if file_name.endswith('.jpeg'):  # Specify the image file formats
            source_path = os.path.join(source_folder, file_name)
            destination_path = os.path.join(destination_folder, folder[0], subfolder) + '/'
            shutil.copy(source_path, destination_path)
            #print(f"Copied: {file_name}")


In [107]:
from ultralytics import YOLO

# Load a model

model = YOLO('yolov8n-cls.pt')  # load a pretrained model (recommended for training)
#model = YOLO('yolov8n-cls.yaml')  # build a new model from YAML
#model = YOLO('yolov8n-cls.pt')  # load a pretrained model (recommended for training)
# model = YOLO('yolov8n-cls.yaml').load('yolov8n-cls.pt')  # build from YAML and transfer weights
# Train the model

results = model.train(data='/content/oct_my_dataset', epochs=2, imgsz=640)

Ultralytics YOLOv8.0.238 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/oct_my_dataset, epochs=2, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

train: Scanning /content/oct_my_dataset/train... 2000 images, 0 corrupt: 100%|██████████| 2000/2000 [00:00<00:00, 3645.14it/s]

train: New cache created: /content/oct_my_dataset/train.cache



val: Scanning /content/oct_my_dataset/val... 32 images, 0 corrupt: 100%|██████████| 32/32 [00:00<00:00, 4758.82it/s]

val: New cache created: /content/oct_my_dataset/val.cache


...

               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:04<00:00,  4.21s/it]

                   all      0.844          1



2 epochs completed in 0.503 hours.
Optimizer stripped from runs/classify/train14/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train14/weights/best.pt, 3.0MB

Validating runs/classify/train14/weights/best.pt...
Ultralytics YOLOv8.0.238 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n-cls summary (fused): 73 layers, 1440004 parameters, 0 gradients, 3.3 GFLOPs
train: /content/oct_my_dataset/train... found 2000 images in 4 classes ✅ 
val: /content/oct_my_dataset/val... found 32 images in 4 classes ✅ 
test: /content/oct_my_dataset/test... found 400 images in 4 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:05<00:00,  5.46s/it]


                   all      0.844          1
Speed: 0.0ms preprocess, 149.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train14
Results saved to runs/classify/train14


In [108]:
# Validate the modelblob:https://colab.research.google.com/f8428635-acf6-400f-b52f-379198002792
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.top1   # top1 accuracy
metrics.top5   # top5 accuracy

Ultralytics YOLOv8.0.238 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n-cls summary (fused): 73 layers, 1440004 parameters, 0 gradients, 3.3 GFLOPs
train: /content/oct_my_dataset/train... found 2000 images in 4 classes ✅ 
val: /content/oct_my_dataset/val... found 32 images in 4 classes ✅ 
test: /content/oct_my_dataset/test... found 400 images in 4 classes ✅ 


val: Scanning /content/oct_my_dataset/val... 32 images, 0 corrupt: 100%|██████████| 32/32 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:04<00:00,  2.28s/it]


                   all      0.844          1
Speed: 0.0ms preprocess, 106.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train142


1.0

In [109]:
# Predict with the model
results = model('/content/oct_my_dataset/test/CNV/CNV-103044-10.jpeg')  # predict on an image


image 1/1 /content/oct_my_dataset/test/CNV/CNV-103044-10.jpeg: 640x640 CNV 0.93, DRUSEN 0.06, DME 0.01, NORMAL 0.00, 226.9ms
Speed: 64.8ms preprocess, 226.9ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
import os
import cv2

%cd /content/OCT2017 /

train_path = "/content/OCT2017 /train/"
#val_path = "/content/OCT2017 /val/"
test_path = "/content/OCT2017 /test/"

classes = ['CNV', 'DME', 'DRUSEN', 'NORMAL']


def load(path,num):
    # List to store images and labels
    listed_paths = []
    images = []
    labels = []

    for i in classes:
        base_path = os.path.join(path, i)
        imgs = os.listdir(base_path)
        for img in imgs[:num]:
            img_path = base_path + '/' + img
            listed_paths.append(img_path)
            image = cv2.imread(img_path)
            image = image[:,:,0]
            image = cv2.resize(image, (256, 256)) #256*256*1 pixels
            images.append(image.flatten())
            labels.append(i)
    return np.array(listed_paths), np.array(images), np.array(labels)

train_paths, train_images, train_labels = load(train_path,1000)
#val_paths, val_images, val_labels = load(val_path,100)
test_paths, test_images, test_labels = load(test_path,200)

# Splitting the data into training and validating sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)



/content/OCT2017 
